In [7]:
import pandas as pd
import requests

In [2]:
# REGIONALES
df_regionales = pd.read_json('regionales.json')
df_regionales.to_csv('partidos_regional.csv')

# PROVINCIALES
df_provinciales = pd.read_json('provinciales.json')
df_provinciales.to_csv('partidos_provincial.csv')

# DISTRITALES
df_distritales = pd.read_json('distritales.json')
df_distritales.to_csv('partidos_distrital.csv')

# DISTRITALES LIMA METROPOLITANA
df_distritales_lima = pd.read_json('distrital_lima_metropolitana.json')
df_distritales_lima.to_csv('partidos_distrital_lima_metropolitana.csv')

In [3]:
# LISTA DE CANDIDATOS
for txCodExpedienteExt in df_distritales_lima['txCodExpedienteExt']:
    url = f'https://apiplataformaelectoral3.jne.gob.pe/api/v1/plan-gobierno/candidatos?IdProcesoElectoral=113&TxCodExpedienteExt={txCodExpedienteExt}'
    resp = requests.get(url=url, verify=False)
    data = resp.json()['data']
    if data != []:
        df = pd.DataFrame(data)
        df['txCodExpedienteExt'] = txCodExpedienteExt
        df.to_csv(f"listas_distritales_lima_metropolitana/{txCodExpedienteExt}.csv")

/home/annaabsi/.local/lib/python3.9/site-packages/urllib3/connectionpool.py:1043: InsecureRequestWarning: Unverified HTTPS request is being made to host 'apiplataformaelectoral3.jne.gob.pe'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(
/home/annaabsi/.local/lib/python3.9/site-packages/urllib3/connectionpool.py:1043: InsecureRequestWarning: Unverified HTTPS request is being made to host 'apiplataformaelectoral3.jne.gob.pe'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(
/home/annaabsi/.local/lib/python3.9/site-packages/urllib3/connectionpool.py:1043: InsecureRequestWarning: Unverified HTTPS request is being made to host 'apiplataformaelectoral3.jne.gob.pe'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnin

In [4]:
# CONCATENAR
import pandas as pd
import glob
import os

# setting the path for joining multiple files
files = os.path.join("listas_distritales_lima_metropolitana", "*.csv")

# list of merged files returned
files = glob.glob(files)

print("Resultant CSV after joining all CSV files at a particular location...");

# joining files with concat and read_csv
df_candidatos = pd.concat(map(pd.read_csv, files), ignore_index=True).drop(columns=['Unnamed: 0'])
df_candidatos.to_csv('candidatos_distrital_lima_metropolitana.csv')
df_candidatos

Resultant CSV after joining all CSV files at a particular location...


,idHojaVida,feNacimiento,txSexo,designado,idProcesoElectoral,numeroDocumento,nombreCompleto,estado,idOrganizacionPolitica,organizacionPolitica,numeroCandidato,rutaArchivo,postulaDepartamento,postulaProvincia,postulaDistrito,cargo,nativo,txCodExpedienteExt
0,149958.0,1976-08-30T00:00:00,F,NaN,0.0,80012731,JULIA IBET TATAJE TABOADA,INSCRITO,2731.0,PODEMOS PERU,0.0,/Assets/Fotos-HojaVida/149958.jpg,LIMA,LIMA,PUNTA NEGRA,['ALCALDE DISTRITAL'],NO,ERM.2022007338
1,149959.0,1957-04-29T00:00:00,M,NaN,0.0,9124435,MARCOS ARTURO PAREDES MONZON,INSCRITO,2731.0,PODEMOS PERU,1.0,/Assets/Fotos-HojaVida/149959.jpg,LIMA,LIMA,PUNTA NEGRA,['REGIDOR DISTRITAL'],NO,ERM.2022007338
2,149960.0,1976-05-03T00:00:00,F,NaN,0.0,9531590,MARIA LUISA ALCEDO HUAMAN,INSCRITO,2731.0,PODEMOS PERU,2.0,/Assets/Fotos-HojaVida/149960.jpg,LIMA,LIMA,PUNTA NEGRA,['REGIDOR DISTRITAL'],NO,ERM.2022007338
3,149961.0,2000-08-24T00:00:00,M,NaN,0.0,75245212,DIEGO CRISTIAN VILCHEZ QUISPE,INSCRITO,2731.0,PODEMOS PERU,3.0,/Assets/Fotos-HojaVida/149961.jpg,LIMA,LIMA,PUNTA NEGRA,['REGIDOR DISTRITAL'],NO,ERM.2022007338
4,149962.0,2001-11-17T00:00:00,F,NaN,0.0,77153739,ESTRELLA EDITH GASTULO TICSE,IMPROCEDENTE,2731.0,PODEMOS PERU,4.0,/Assets/Fotos-HojaVida/149962.jpg,LIMA,LIMA,PUNTA NEGRA,['REGIDOR DISTRITAL'],NO,ERM.2022007338
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5409,140229.0,1980-12-06T00:00:00,M,NaN,0.0,42016577,CARLOS JUNCO MINAYA,INSCRITO,22.0,RENOVACION POPULAR,7.0,/Assets/Fotos-HojaVida/140229.jpg,LIMA,LIMA,LURIGANCHO,['REGIDOR DISTRITAL'],NO,ERM.2022012663
5410,140230.0,1968-07-16T00:00:00,F,NaN,0.0,21860728,SANTA ANA MARTINEZ GONZALES,INSCRITO,22.0,RENOVACION POPULAR,8.0,/Assets/Fotos-HojaVida/140230.jpg,LIMA,LIMA,LURIGANCHO,['REGIDOR DISTRITAL'],NO,ERM.2022012663
5411,240891.0,1954-10-22T00:00:00,M,NaN,0.0,9193629,DONATO CRISOSTOMO CUETO,INSCRITO,22.0,RENOVACION POPULAR,9.0,/Assets/Fotos-HojaVida/240891.jpg,LIMA,LIMA,LURIGANCHO,['REGIDOR DISTRITAL'],NO,ERM.2022012663
5412,140232.0,2000-10-17T00:00:00,F,NaN,0.0,75663791,ANGELLI ALEXANDRA CHOQUE QUISPE,INSCRITO,22.0,RENOVACION POPULAR,10.0,/Assets/Fotos-HojaVida/140232.jpg,LIMA,LIMA,LURIGANCHO,['REGIDOR DISTRITAL'],NO,ERM.2022012663


In [3]:
# MERGE
df1 = pd.read_csv('candidatos_regional.csv')
df2 = pd.read_csv('partidos_regional.csv')
df_merged = df1.merge(df2, how='inner', on='txCodExpedienteExt').drop(columns=['Unnamed: 0_x', 'designado', 'idProcesoElectoral', 'Unnamed: 0_y', 'txTipoPlan', 'txDistritoElectoral'])
df_merged.to_csv('base_datos_regional_final_02oct.csv')

df1 = pd.read_csv('candidatos_provincial.csv')
df2 = pd.read_csv('partidos_provincial.csv')
df_merged = df1.merge(df2, how='inner', on='txCodExpedienteExt').drop(columns=['Unnamed: 0_x', 'designado', 'idProcesoElectoral', 'Unnamed: 0_y', 'txTipoPlan', 'txDistritoElectoral'])
df_merged.to_csv('base_datos_provincial_final_02oct.csv')

df1 = pd.read_csv('candidatos_distrital.csv')
df2 = pd.read_csv('partidos_distrital.csv')
df_merged = df1.merge(df2, how='inner', on='txCodExpedienteExt').drop(columns=['Unnamed: 0_x', 'designado', 'idProcesoElectoral', 'Unnamed: 0_y', 'txTipoPlan', 'txDistritoElectoral'])
df_merged.to_csv('base_datos_distrital_final_02oct.csv')

/tmp/ipykernel_40252/2671544210.py:12: DtypeWarning: Columns (4) have mixed types. Specify dtype option on import or set low_memory=False.
  df1 = pd.read_csv('candidatos_distrital.csv')


In [6]:
df1 = pd.read_csv('candidatos_distrital_lima_metropolitana.csv')
df2 = pd.read_csv('partidos_distrital_lima_metropolitana.csv')
df_merged = df1.merge(df2, how='inner', on='txCodExpedienteExt').drop(columns=['Unnamed: 0_x', 'designado', 'idProcesoElectoral', 'Unnamed: 0_y', 'txTipoPlan', 'txDistritoElectoral'])
df_merged.to_csv('base_datos_distrital_lima_metropolitana_final_02oct.csv')

In [19]:
# solo gobernadores y vice en regionales
df_gobernadores_vicegobernadores_regional = pd.read_csv('base_datos_regional_final.csv')
df_gobernadores_vicegobernadores_regional = df_gobernadores_vicegobernadores_regional[df_gobernadores_vicegobernadores_regional['cargo'].str.contains('GOBERNADOR REGIONAL' or 'VICEGOBERNADOR REGIONAL')].drop(columns=['Unnamed: 0'])
df_gobernadores_vicegobernadores_regional.cargo = df_gobernadores_vicegobernadores_regional.cargo.apply(lambda x: 'VICEGOBERNADOR REGIONAL' if 'VICEGOBERNADOR REGIONAL' in x else 'GOBERNADOR REGIONAL')
df_gobernadores_vicegobernadores_regional.to_csv('solo_gobernadores_y_vicegobernadores.csv', index=False)

In [3]:
# solo alcaldes provinciales
df_alcaldes_provincial = pd.read_csv('base_datos_provincial_final.csv')
df_alcaldes_provincial = df_alcaldes_provincial[df_alcaldes_provincial['cargo'].str.contains('ALCALDE PROVINCIAL')].drop(columns=['Unnamed: 0'])
df_alcaldes_provincial['cargo'] = 'ALCALDE PROVINCIAL'
df_alcaldes_provincial.to_csv('solo_alcaldes_provinciales.csv', index=False)

In [8]:
# solo alcaldes distrital Lima Metropolitana
df_alcaldes_distrital = pd.read_csv('base_datos_distrital_lima_metropolitana_final_02oct.csv')
df_alcaldes_distrital = df_alcaldes_distrital[df_alcaldes_distrital['cargo'].str.contains('ALCALDE DISTRITAL')].drop(columns=['Unnamed: 0'])
df_alcaldes_distrital['cargo'] = 'ALCALDE DISTRITAL'
df_alcaldes_distrital.to_csv('solo_alcaldes_distritales_lima_metropolitana.csv', index=False)

In [ ]:
# Numero de partidos: 147
df2 = pd.read_csv('partidos_general.csv')
numero_partidos = df2['txOrganizacionPolitica'].nunique()
numero_partidos

In [ ]:
# Numero de apellidos unicos: 107330 de 135056 DNIs
df3 = pd.read_csv('base_datos_final.csv')
df3['Apellidos'] = (df3['txNombreCompleto'].str.split().str[-2:].str.join(' '))
personas_unicas = df3['txDocumentoIdentidad'].nunique()
apellidos_unicos = df3['Apellidos'].nunique()
df3.to_csv('base_datos_final.csv')

In [ ]:
# DNI duplicados: 369 registros, 184 personas
dni_duplicados = df3[df3['txDocumentoIdentidad'].duplicated(keep=False)]
dni_duplicados.to_csv('dni_duplicados.csv')
personas_unicas = dni_duplicados['txDocumentoIdentidad'].nunique()
personas_unicas
moda = dni_duplicados['txDocumentoIdentidad'].mode()
moda

In [ ]:
# Elecciones regionales: gobernadores y vice gobernadores
df_regionales_gobernadores = pd.read_csv('base_datos_final.csv')
df_regionales_gobernadores = df_regionales_gobernadores[df_regionales_gobernadores['txCargoEleccion']=='GOBERNADOR REGIONAL'].reset_index()
df_regionales_gobernadores.to_csv('gobernadores_regionales.csv')
df_regionales_vicegobernadores = pd.read_csv('base_datos_final.csv')
df_regionales_vicegobernadores = df_regionales_vicegobernadores[df_regionales_vicegobernadores['txCargoEleccion']=='VICEGOBERNADOR REGIONAL'].reset_index()
df_regionales_vicegobernadores.to_csv('vicegobernadores_regionales.csv')

In [ ]:
# Elecciones provinciales: alcaldes
df_provinciales_alcaldes = pd.read_csv('base_datos_final.csv')
df_provinciales_alcaldes = df_provinciales_alcaldes[df_provinciales_alcaldes['txCargoEleccion']=='ALCALDE PROVINCIAL'].reset_index()
df_provinciales_alcaldes.to_csv('alcaldes_provinciales.csv')